# Generating a Vocabulary Function with Spacy whilst modelling

If sample size is less than 10, create more features
Adjust number of features depending on the size of our training data

Create the vocabulary using Spacy Tokenizer in our
Fit using the same tokenizer for gridsearched model to cut down on processing time

In [9]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.base import TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
from sklearn.feature_extraction.stop_words import ENGLISH_STOP_WORDS
from sklearn.metrics import accuracy_score
from nltk.corpus import stopwords
import string
import re
from spacy.en import English
parser = English()
import pickle
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import os
import pandas as pd
import numpy as np
import time

In [10]:
cv = CountVectorizer()

In [11]:
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import f1_score

//anaconda/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
//anaconda/lib/python2.7/site-packages/sklearn/grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [12]:
# DEFINE DIRECTORY PATH
path_to_json = './TrainingData2014/'

# CREATE LIST OF FILES FROM THE DIRECTORY
json_files = [pos_json for pos_json in os.listdir(path_to_json) if pos_json.endswith('.json')]

# DEFINE PANDAS DATAFRAME
df = pd.DataFrame()

# LOOP THROUGH FILES, READ IN JSON AND BUILD DATAFRAME
for index, js in enumerate(json_files):
    json_data = pd.read_json(os.path.join(path_to_json, js), 'r')
    df = df.append(json_data)
    
# LOOK AT TOPIC DICTIONARY AND GET A TOPIC COUNT
topic_file = open('./topicDictionary.txt', 'r')
topics = topic_file.read().split('\r\n')

# SPLITTING THE ELEMENTS OF THE JSON INTO TEXT, PUBLICATION DATE AND TOPICS
df['text'] = df.TrainingData.apply(lambda x: x['bodyText'])
df['pubdate'] = df.TrainingData.apply(lambda x: x['webPublicationDate'])
df['topics'] = df.TrainingData.apply(lambda x: x['topics'])

# DROP FIRST TWO COLUMNS
df.reset_index(inplace=True, drop=True)
df.drop('TrainingData', axis=1, inplace=True)

# DEFINE FUNCTION TO CREATE OUR DATAFRAME
def topic_col(x):
    a = 0
    for elem in x:
        if elem == topic:
            a = 1
    return a

# RUN TOPIC COL FUNCTION ON ALL DATA
for topic in topics:
    time1 = time.time()
    df[topic] = df['topics'].map(topic_col)
    print topic
    time2 = time.time()
    time_in_s = (time2-time1)
    print 'Function takes around %0.3f seconds to run' % (time_in_s)

activism
Function takes around 0.336 seconds to run
afghanistan
Function takes around 0.268 seconds to run
aid
Function takes around 0.229 seconds to run
algerianhostagecrisis
Function takes around 0.294 seconds to run
alqaida
Function takes around 0.235 seconds to run
alshabaab
Function takes around 0.318 seconds to run
antiwar
Function takes around 0.310 seconds to run
arabandmiddleeastprotests
Function takes around 0.268 seconds to run
armstrade
Function takes around 0.209 seconds to run
australianguncontrol
Function takes around 0.241 seconds to run
australiansecurityandcounterterrorism
Function takes around 0.344 seconds to run
bastilledaytruckattack
Function takes around 0.262 seconds to run
belgium
Function takes around 0.250 seconds to run
berlinchristmasmarketattack
Function takes around 0.298 seconds to run
bigdata
Function takes around 0.251 seconds to run
biometrics
Function takes around 0.251 seconds to run
bokoharam
Function takes around 0.279 seconds to run
bostonmaratho

undercoverpoliceandpolicing
Function takes around 0.129 seconds to run
unitednations
Function takes around 0.173 seconds to run
usguncontrol
Function takes around 0.278 seconds to run
values
Function takes around 0.110 seconds to run
warcrimes
Function takes around 0.118 seconds to run
warreporting
Function takes around 0.127 seconds to run
weaponstechnology
Function takes around 0.122 seconds to run
womeninbusiness
Function takes around 0.138 seconds to run
woolwichattack
Function takes around 0.101 seconds to run
worldmigration
Function takes around 0.120 seconds to run
zikavirus
Function takes around 0.102 seconds to run


In [14]:
# CREATE FINAL TEST DF
testdf = pd.read_json('/Users/sudheerpamula/Downloads/TestData.json', 'r')
testdf['text'] = testdf.TestData.apply(lambda x: x['bodyText'])
testdf['pubdate'] = testdf.TestData.apply(lambda x: x['webPublicationDate'])
testdf['topics'] = testdf.TestData.apply(lambda x: x['topics'])
testdf.drop('TestData', axis=1, inplace=True)

In [20]:
# DEFINE SUBMISSION CSV DATAFRAME
submission = pd.DataFrame(index=testdf.index, columns=topics)

In [22]:
def sampler(topic, df):
    
    # SELECT INDICES OF TOPIC ARTICLES
    topicindexes = df[df[topic] == 1].index.tolist()
    
    # FIND COUNT OF TOPIC ARTICLES
    articlecount = len(topicindexes)
    
    if articlecount < 10:
        # SELECT NON TOPIC INDICES
        nontopicarticlesindexes = df[df[topic] == 0].sample(articlecount*27, random_state=42).index.tolist()
        # CREATE LIST OF COMBINED INDICES
        sampleindex = topicindexes + nontopicarticlesindexes 
        # CREATE NEW DATAFRAME
        X = df.iloc[sampleindex]['text'].reset_index(drop=True)
        y = df.iloc[sampleindex][topic].reset_index(drop=True)
        X = X.append(df.iloc[topicindexes]['text'])
        y = y.append(df.iloc[topicindexes][topic])
        X = X.append(df.iloc[topicindexes]['text'])
        y = y.append(df.iloc[topicindexes][topic])
        X = X.append(df.iloc[topicindexes]['text']).reset_index(drop=True)
        y = y.append(df.iloc[topicindexes][topic]).reset_index(drop=True)
        
    elif articlecount < 100:
        # SELECT NON TOPIC INDICES
        nontopicarticlesindexes = df[df[topic] == 0].sample(articlecount*9, random_state=42).index.tolist()
        # CREATE LIST OF COMBINED INDICES
        sampleindex = topicindexes + nontopicarticlesindexes 
        # CREATE NEW DATAFRAME
        X = df.iloc[sampleindex]['text'].reset_index(drop=True)
        y = df.iloc[sampleindex][topic].reset_index(drop=True)
        X = X.append(df.iloc[topicindexes]['text']).reset_index(drop=True)
        y = y.append(df.iloc[topicindexes][topic]).reset_index(drop=True)
        
    else:
        # SELECT NON TOPIC INDICES
        topicindexes = df[df[topic] == 1].sample(100).index.tolist()
        
        nontopicarticlesindexes = df[df[topic] == 0].sample(len(topicindexes)*9, random_state=42).index.tolist()
        nonarticlecount = len(nontopicarticlesindexes)
        # CREATE LIST OF COMBINED INDICES
        sampleindex = topicindexes + nontopicarticlesindexes
        # CREATE NEW DATAFRAME
        X = df.iloc[sampleindex]['text'].reset_index(drop=True)
        y = df.iloc[sampleindex][topic].reset_index(drop=True)
    
    return X, y

In [23]:
# ADDED 4 THAT WERE NOT IN 2014 ARTICLES
topics_0 = ['activism', 'turkeycoupattempt', 'tunisiaattack2015', 'sanbernardinoshooting', 'peaceandreconciliation',
            'parisattacks', 'orlandoterrorattack', 'munichshooting', 'francetrainattack', 'charliehebdoattack',
            'brusselsattacks', 'berlinchristmasmarketattack','bastilledaytruckattack','zikavirus', 'jubilee', 'values', 'rio20earthsummit', 'algerianhostagecrisis']

In [24]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.base import TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
from sklearn.feature_extraction.stop_words import ENGLISH_STOP_WORDS
from sklearn.metrics import accuracy_score
from nltk.corpus import stopwords
import string
import re

# A custom stoplist
STOPLIST = set(stopwords.words('english') + ["n't", "'s", "'m", "ca"] + list(ENGLISH_STOP_WORDS))
# List of symbols we don't care about
SYMBOLS = " ".join(string.punctuation).split(" ") + ["-----", "---", "...", "“", "”", "'ve"]

# Every step in a pipeline needs to be a "transformer". Define a custom transformer to clean text using spaCy
class CleanTextTransformer(TransformerMixin):
    """
    Convert text to cleaned text
    """

    def transform(self, X, **transform_params):
        return [cleanText(text) for text in X]

    def fit(self, X, y=None, **fit_params):
        return self

    def get_params(self, deep=True):
        return {}
    
# A custom function to clean the text before sending it into the vectorizer
def cleanText(text):
    # get rid of newlines
    text = text.strip().replace("\n", " ").replace("\r", " ")
    
    # replace twitter @mentions
    mentionFinder = re.compile(r"@[a-z0-9_]{1,15}", re.IGNORECASE)
    text = mentionFinder.sub("@MENTION", text)
    
    # replace HTML symbols
    text = text.replace("&amp;", "and").replace("&gt;", ">").replace("&lt;", "<")
    
    # lowercase
    text = text.lower()

    return text

# A custom function to tokenize the text using spaCy
# and convert to lemmas
def tokenizeText(sample):

    # get the tokens using spaCy
    tokens = parser(sample)

    # lemmatize
    lemmas = []
    for tok in tokens:
        lemmas.append(tok.lemma_.lower().strip() if tok.lemma_ != "-PRON-" else tok.lower_)
    tokens = lemmas

    # stoplist the tokens
    tokens = [tok for tok in tokens if tok not in STOPLIST]

    # stoplist symbols
    tokens = [tok for tok in tokens if tok not in SYMBOLS]

    # remove large strings of whitespace
    while "" in tokens:
        tokens.remove("")
    while " " in tokens:
        tokens.remove(" ")
    while "\n" in tokens:
        tokens.remove("\n")
    while "\n\n" in tokens:
        tokens.remove("\n\n")

    return tokens

def printNMostInformative(vectorizer, clf, N):
    """Prints features with the highest coefficient values, per class"""
    feature_names = vectorizer.get_feature_names()
    coefs_with_fns = sorted(zip(clf.coef_[0], feature_names))
    topClass1 = coefs_with_fns[:N]
    topClass2 = coefs_with_fns[:-(N + 1):-1]
    print("Worst Features: ")
    for feat in topClass1:
        print(feat)
        spacyworstvocab[topic] = feat
    print("Strongest Features: ")
    for feat in topClass2:
        print(feat)
        spacyvocab[topic] = feat

In [25]:
parameters = {
    'vectorizer__max_df': (0.15, 0.25),
    'tfidf__norm': ('l2', 'l1'),
    'clf__penalty': ('l1', 'l2'),
    'vectorizer__binary': (True, False),
    'clf__C': (0.001, 0.01, 0.1, 1, 10, 100, 1000),
}

In [26]:
spacyvocab = {}
spacyworstvocab = {}

In [27]:
vectorizer = CountVectorizer(tokenizer=tokenizeText, ngram_range=(1,2))

clf = LogisticRegression()

pipe = Pipeline([('cleanText', CleanTextTransformer()),
                 ('vectorizer', vectorizer), 
                 ('tfidf', TfidfTransformer()), 
                 ('ss', StandardScaler(with_mean=False)), 
                 ('clf', clf)])

for topic in topics:
    if topic in topics_0:
        submission[topic] = 0
    
    else:
        X, y = sampler(topic, df)
        train, test, labelsTrain, labelsTest = train_test_split(X, y, test_size=0.25, stratify=y, random_state=42)
        
        pipe.fit(train, labelsTrain)

        grid_search = GridSearchCV(pipe, parameters, n_jobs=-1, verbose=1)
        grid_search.fit(train, labelsTrain)

        preds = grid_search.best_estimator_.predict(test)
        topicf1score = f1_score(labelsTest, preds)
        print("----------------------------------------------------------------------------------------------")
        print topic, "F1 score:", topicf1score
        print "Grid Search Best Params:", grid_search.best_estimator_.get_params
        
        # PICKLE GRID SEARCH BEST PARAMETERS
        with open('Pickles/gs_logreg'+str(topic)+'.pickle', 'wb') as handle:
            pickle.dump(spacyvocab, handle, protocol=pickle.HIGHEST_PROTOCOL)
        
        # MAKE FINAL PREDICTIONS
        preds_final = grid_search.best_estimator_.predict(testdf['text'])
        
        # WRITE TO CSV
        submission[topic] = preds_final
        submission.to_csv('submission_spacytokenizer_2.csv')
        print("----------------------------------------------------------------------------------------------")
        print("Predictions Completed on Test Data")
        
        print("----------------------------------------------------------------------------------------------")
        print("Top 10 features used to predict: ")
        printNMostInformative(vectorizer, clf, 30)
        
        print("----------------------------------------------------------------------------------------------")
        print("Pickled Vocabulary to File: ", topic)
        with open('spacyvocab.pickle', 'wb') as handle:
            pickle.dump(spacyvocab, handle, protocol=pickle.HIGHEST_PROTOCOL)
        with open('spacyworstvocab.pickle', 'wb') as handle:
            pickle.dump(spacyworstvocab, handle, protocol=pickle.HIGHEST_PROTOCOL)

/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:65: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal


Fitting 3 folds for each of 112 candidates, totalling 336 fits


[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed: 20.1min


KeyboardInterrupt: 

In [28]:
topics

['activism',
 'afghanistan',
 'aid',
 'algerianhostagecrisis',
 'alqaida',
 'alshabaab',
 'antiwar',
 'arabandmiddleeastprotests',
 'armstrade',
 'australianguncontrol',
 'australiansecurityandcounterterrorism',
 'bastilledaytruckattack',
 'belgium',
 'berlinchristmasmarketattack',
 'bigdata',
 'biometrics',
 'bokoharam',
 'bostonmarathonbombing',
 'britisharmy',
 'brusselsattacks',
 'cameroon',
 'carers',
 'charliehebdoattack',
 'chemicalweapons',
 'clusterbombs',
 'cobra',
 'conflictanddevelopment',
 'controversy',
 'criminaljustice',
 'cybercrime',
 'cyberwar',
 'darknet',
 'dataprotection',
 'debate',
 'defence',
 'deflation',
 'drones',
 'drugs',
 'drugspolicy',
 'drugstrade',
 'earthquakes',
 'ebola',
 'economy',
 'egypt',
 'encryption',
 'energy',
 'espionage',
 'ethics',
 'europeanarrestwarrant',
 'europeancourtofhumanrights',
 'events',
 'extradition',
 'famine',
 'farright',
 'firefighters',
 'forensicscience',
 'france',
 'francetrainattack',
 'freedomofspeech',
 'genevaconv